# Environment

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.offline import init_notebook_mode, iplot
from plotly.graph_objs import *
from plotly.subplots import make_subplots
import plotly.figure_factory as ff

In [3]:
# Basic
import os
import gc
import re
import sys
import glob
import numpy as np
import pandas as pd
from pathlib import Path
from pprint import pprint
import datetime

from collections import defaultdict
from typing import (List, Dict, Any, NoReturn, 
                    Tuple, Optional, Union)

from tqdm import tqdm, tqdm_notebook
import warnings
warnings.filterwarnings('ignore')

In [6]:
sys.path.insert(0,"..")

from config import init_config, config
from helpers import read_json, write_json, update_parameters
from eyemovements.filtering import sgolay_filter_dataset
from eyemovements.eyemovements_utils import (GazeState, get_movement_indexes)
from eyemovements.eyemovements_classifier import IVDT
from eyemovements.classification import classify_eyemovements_wrapper, run_eyemovements_classification
from eyemovements.eyemovements_metrics import estimate_quality
from create_training_dataset import TrainDataset

In [7]:
# System adjustments - for all colums to fit into output (default width is 80)
pd.options.display.width = 2500
pd.options.display.max_rows = 999
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', None)
pd.options.display.max_colwidth = 100
sns.set_style("whitegrid")

## Define paths

In [8]:
CONFIG_FILE = "../set_locations.ini"
init_config(CONFIG_FILE)
TRAIN_DIR = config.get("DataPaths", "train_data")
OWNER_DIR = config.get("DataPaths", "owner_data")
RUN_DIR = config.get("DataPaths", "run_data")

# Load dataset

In [9]:
dataset = TrainDataset(config.get("DataPaths", "run_data")).create_dataset()

100%|██████████████████████████████████████████████████████████████████████████████████| 14/14 [00:01<00:00,  8.25it/s]


## Eye movement classification (I-VDT) parameters

In [10]:
curr_model_params = dict(read_json(config.get('EyemovementClassification', 'model_params')))
pprint(curr_model_params)

{'dispersion_threshold': 5,
 'max_saccade_duration_threshold': 0.001,
 'min_fixation_duration_threshold': 0.001,
 'min_saccade_duration_threshold': 0.0002,
 'min_sp_duration_threshold': 0.002,
 'saccade_min_velocity': 4.5,
 'window_size': 10}


## Parameters search

In [11]:
cls_dataset = classify_eyemovements_wrapper(dataset)

100%|██████████████████████████████████████████████████████████████████████████████████| 12/12 [00:00<00:00, 62.14it/s]

Resulted list of data length: 12



  0%|                                                                                           | 0/12 [00:00<?, ?it/s]


User id: 0
Stimulus type: cat_s#1


 33%|███████████████████████████▋                                                       | 4/12 [00:00<00:01,  4.18it/s]

[INFO-FILTER]: found too small sp: [10]!
[INFO-FILTER]: found too small sp: [23]!

User id: 0
Stimulus type: cat_f#1
[INFO-FILTER]: found too small sp: [213]!

User id: 1
Stimulus type: cat_f#1

User id: 1
Stimulus type: cat_f#1

User id: 1
Stimulus type: cat_s#1


 58%|████████████████████████████████████████████████▍                                  | 7/12 [00:01<00:01,  3.97it/s]


User id: 1
Stimulus type: cat_f#1

User id: 2
Stimulus type: cat_s#1
[INFO-FILTER]: found too small sp: [18]!
[INFO-FILTER]: found too small sp: [87]!
[INFO-FILTER]: found too small sp: [98]!
[INFO-FILTER]: found too small sp: [195]!
[INFO-FILTER]: found too small sp: [394]!
[INFO-FILTER]: found too small sp: [399]!
[INFO-FILTER]: found too small sp: [404]!
[INFO-FILTER]: found too small sp: [419]!
[INFO-FILTER]: found too small sp: [519]!
[INFO-FILTER]: found too small sp: [594]!
[INFO-FILTER]: found too small sp: [647]!
[INFO-FILTER]: found too small sp: [650]!
[INFO-FILTER]: found too small sp: [707]!
[INFO-FILTER]: found too small sp: [822]!
[INFO-FILTER]: found too small sp: [935]!
[INFO-FILTER]: found too small sp: [1141]!
[INFO-FILTER]: found too small sp: [1214]!
[INFO-FILTER]: found too small sp: [1254]!
[INFO-FILTER]: found too small sp: [1826]!
[INFO-FILTER]: found too small sp: [1890]!
[INFO-FILTER]: found too small sp: [1901]!
[INFO-FILTER]: found too small sp: [1941]!
[I

100%|██████████████████████████████████████████████████████████████████████████████████| 12/12 [00:01<00:00,  9.03it/s]


User id: 7
Stimulus type: cat_s#1
[INFO-FILTER]: found too small sp: [0, 1, 2, 3]!


In [12]:
cls_dataset[0].head()

,timestamps,stim_X,stim_Y,gaze_X,gaze_Y,frame,timestamp,success,session_id,stimulus_type,user_id,x_diff,y_diff,filename,filtered_X,filtered_Y,velocity_X,velocity_Y,velocity_sqrt,stimulus_velocity,acceleration_X,acceleration_sqrt,movements,movements_type
0,1.608139e+09,449,121,977,555,6,1970-01-01 00:00:00.167,1,0,cat_s#1,0,-528,-434,C:\Users\airen\Projects\EyeGazeTesting\Verification Task\GazeVerificationStand\data\run\exp-2020...,701.809518,789.204374,-7.792055,-3.836600,8.685368,0.239858,-0.082268,0.133076,2,saccade
1,1.608139e+09,449,121,930,534,6,1970-01-01 00:00:00.167,1,0,cat_s#1,0,-481,-413,C:\Users\airen\Projects\EyeGazeTesting\Verification Task\GazeVerificationStand\data\run\exp-2020...,692.847905,779.356588,-15.814290,-14.145071,21.217323,0.124111,-2.590376,3.615771,2,saccade
2,1.608139e+09,449,121,891,516,7,1970-01-01 00:00:00.200,1,0,cat_s#1,0,-442,-395,C:\Users\airen\Projects\EyeGazeTesting\Verification Task\GazeVerificationStand\data\run\exp-2020...,669.691450,753.811130,-20.998389,-21.223874,29.856074,0.540354,-3.366060,4.662716,2,saccade
3,1.608139e+09,449,121,859,501,7,1970-01-01 00:00:00.200,1,0,cat_s#1,0,-410,-380,C:\Users\airen\Projects\EyeGazeTesting\Verification Task\GazeVerificationStand\data\run\exp-2020...,641.549441,722.503270,-23.121678,-24.367547,33.591507,1.023571,-1.874189,2.532831,2,saccade
4,1.608139e+09,455,121,834,487,7,1970-01-01 00:00:00.200,1,0,cat_s#1,0,-379,-366,C:\Users\airen\Projects\EyeGazeTesting\Verification Task\GazeVerificationStand\data\run\exp-2020...,618.060772,695.935814,-21.255685,-22.777902,31.155047,1.402266,1.016104,1.547213,2,saccade


In [13]:
cls_dataset[0].movements_type.value_counts()

sp         1638
saccade      90
unknown       2
Name: movements_type, dtype: int64

## Visualization

In [16]:
rus_eng_names = {"fixation": "Фиксация", "saccade": "Саккада", 
                 "sp": "Следящее движение", "unknown": "Неизвестно", "error": "Ошибка трекера"}

In [21]:
def visualize_eyemovements(data: pd.DataFrame,
                       x_col: str="x", y_col: str="y",
                       time_col: str='timestamps',
                       color: str="movement_name"):
    assert ((x_col in data.columns) and (y_col in data.columns)
            and (color in data.columns) and (time_col in data.columns))
    
    color_mapping = dict(read_json(os.path.join(sys.path[0], "settings", "color_mappings.json")))
    data['color'] = data[color].apply(lambda x: color_mapping.get(x, "black"))
    names_mapping = dict(read_json(os.path.join(sys.path[0], "settings", "eng_rus_names.json")))
    data['rus_movements'] = data[color].apply(lambda x: names_mapping.get(x, "black"))

    fig = make_subplots(
        rows=2, cols=2,
        shared_xaxes=False,
        vertical_spacing=0.06,
        specs=[[{}, {}],
               [{"colspan": 2}, None]],
        row_heights=[0.4, 0.6],
        subplot_titles=("Взгляд, координата Х", "Взгляд, координата Y", "Взгляд, координаты X-Y")
    )

    min_ts = np.min(data[time_col])
    for movement_type, df in data.groupby(by='rus_movements'):
        fig.add_trace(go.Scatter(x=df[time_col] - min_ts,
                                 y=df[x_col],
                                 mode='markers',
                                 marker_color=df['color'],
                                 name=movement_type,
                                 showlegend=False), row=1, col=1)

        fig.add_trace(go.Scatter(x=df[time_col] - min_ts,
                                 y=df[y_col],
                                 mode='markers',
                                 marker_color=df['color'],
                                 name=movement_type,
                                 showlegend=False), row=1, col=2)

        fig.add_trace(go.Scatter(x=df[x_col],
                                 y=df[y_col],
                                 mode='markers',
                                 marker_color=df['color'],
                                 name=movement_type), row=2, col=1)

    fig.update_traces(mode='markers', marker_line_width=0.1, marker_size=4)
    fig.update_layout(height=800, width=1000,
                      title_text="Классификация движений глаз")

    fig.update_layout(legend_title_text='Типы движений глаз',
                      legend=dict(font=dict(family="Arial", size=12)))
    fig.update_layout(showlegend=True)
    fig.show()

In [22]:
for i, df in enumerate(cls_dataset):
    print(f"\n\nSession #{i}")
    visualize_eyemovements(df, x_col="gaze_X", y_col="gaze_Y",
                       time_col='timestamps',
                       color='movements_type')
    if i > 1:
        break



Session #0




Session #1




Session #2


## Update parameters

In [15]:
update_parameters(fn=config.get('EyemovementClassification', 'model_params'),
                 parameter_name='saccade_min_velocity', new_value=4.5)